# AzureML Deployment Example

Demonstrates registering and deploying the model to AzureML (conceptual example).

In [1]:
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
import os

## Connect to Workspace

In [2]:
ws = Workspace.from_config()

## Register Model

In [3]:
model = Model.register(workspace=ws,
                      model_path='../models/trained/model.joblib',
                      model_name='bridge_failure_rf')

## Define Inference Environment and Config

In [4]:
env = Environment.from_conda_specification(name='bridge-env', file_path='../environment.yml')
inference_config = InferenceConfig(entry_script='../src/serve/score.py', environment=env)

## Deploy to ACI (Azure Container Instances)

In [5]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)
service = Model.deploy(workspace=ws,
                      name='bridge-failure-api',
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=deployment_config,
                      overwrite=True)
service.wait_for_deployment(show_output=True)
print('Service state:', service.state)